In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import handcalcs.render
from math import sqrt, pi
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sympy
import numpy as np
import forallpeople
from ipycanvas import Canvas, RoughCanvas
import ipywidgets as widgets
from IPython.display import display, HTML
from __future__ import print_function
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import drawSvg as draw
%matplotlib inline
sns.set_theme()

forallpeople.environment('Structural', top_level=True)
kNm = 1000*Nm
KN = 1000*N

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bearing-stiffener-design-to-BS5400:3" data-toc-modified-id="Bearing-stiffener-design-to-BS5400:3-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bearing stiffener design to BS5400:3</a></span><ul class="toc-item"><li><span><a href="#Tension-field-action-" data-toc-modified-id="Tension-field-action--1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Tension field action <a name="a"></a></a></span><ul class="toc-item"><li><span><a href="#British-standard-method" data-toc-modified-id="British-standard-method-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>British standard method</a></span></li><li><span><a href="#Eurocode-method" data-toc-modified-id="Eurocode-method-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Eurocode method</a></span></li></ul></li><li><span><a href="#Destabilising-effect-of-the-web" data-toc-modified-id="Destabilising-effect-of-the-web-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Destabilising effect of the web</a></span></li><li><span><a href="#Axial-force-due-to-transfer-of-load-through-a-cross-frame/beam;" data-toc-modified-id="Axial-force-due-to-transfer-of-load-through-a-cross-frame/beam;-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Axial force due to transfer of load through a cross frame/beam;</a></span></li></ul></li></ul></div>

In [3]:
# import handcalcs.render
# from math import sqrt, pi
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# import sympy
# import numpy as np
# import forallpeople
# from ipycanvas import Canvas, RoughCanvas
# import ipywidgets as widgets
# from IPython.display import display, HTML
# from __future__ import print_function
# from ipywidgets import widgets, interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
# import drawSvg as draw
# %matplotlib inline
# sns.set_theme()

# CSS = """
# .output {
#     align-items: center;
# }

# """

# HTML('<style>{}</style>'.format(CSS))

# Bearing stiffener design to BS5400:3

In accordance with Cl 9.13.3.1, there are a number of forces to consider when designing a transverse stiffener.  

1.1 axial force due to tension field action in accordance with 9.13.3.2;   $F_{tw}$ = {{F_tw}}  
1.2 axial force representing the destabilising influence of the web in accordance with 9.13.3.3;   $F_{wi}$ = {{F_wi2}} kN  
1.3 axial force due to transfer of load through a cross frame/beam;  
1.4 axial force due to a load applied at flange level;  
1.5 axial force due to curvature of flange in accordance with 9.12.3.4;  
1.6 axial force due to change of slope of flange;  
1.7 bending moment due to axial forces above acting eccentrically.

Firstly, define some variables to be used.

Stresses from section 4.3

In [4]:
%%render params 1

sigma_Tweb = (-281*MPa) # tensile stress
sigma_Bweb = (248*MPa) # compressive stress
tau = (6000*1000/(25*1850))*MPa # shear stress




<IPython.core.display.Latex object>

Geometrical and material properties of panel

In [5]:
%%render 1
#short

E = (205000*MPa) #Young's modulus of steel
h = ((1850*mm).prefix('m')) #Height of web
t_w = (25*mm) #Thickness of web
a = ((3000*mm).prefix('m')) # panel length
b = h #Panel width i.e. height of web
l_s = b # length of transverse stiffener 





<IPython.core.display.Latex object>

The neutral axis is found by using similar triangles:

In [6]:
%%render params 1
NA = (h*abs(sigma_Bweb)/(abs(sigma_Tweb) + abs(sigma_Bweb)))  #from bottom

<IPython.core.display.Latex object>

The effective width of the web extends by 16$t_w$ beyond each extreme stiffener.  

Below is a table summarising the section properties of the stiffened bearing area. Parallel axis theorem has been used in order to find the Second moment of Areas about axes X and Y, shown in Figure 1 of BS5400:3.

In [7]:


df = pd.DataFrame({'name': ['Inner 1', 'Outer 2', 'Brg Inner 3', 'Brg Outer 4', 'Inner 5', 'Outer 6'],
                   'B': [25,25,30,30,25,25],
                   'D': [215,215,215,215,215,215],
                   'x': [-300,-300,0,0,300,300],
                   'y': [107.5+float(t_w)/2,-107.5-float(t_w)/2,107.5+float(t_w)/2,-107.5-float(t_w)/2,107.5+float(t_w)/2,-107.5-float(t_w)/2],      

                   })

Web_width = ((abs(df['x'].min()) + abs(df['x'].max()))*mm + 32*t_w).prefix('m')

df2 = pd.DataFrame({'name': ['Web'],
                   'B': [float(Web_width)],
                   'D': [float(t_w)],
                   'x': [0],
                   'y': [0],      
                    
                   })

df = pd.concat([df,df2])
df = df.reset_index(drop=True)

# df = df.apply(pd.to_numeric) # convert all columns of DataFrame

df['Area'] = df['B']*df['D']
df['Ax'] = df['Area']*df['x']
df['Ay'] = df['Area']*df['y']



xNA = df.Ax.sum()/df.Area.sum()
yNA = df.Ay.sum()/df.Area.sum()
A = df.Area.sum()

df.loc["Total", "Area"] = df.Area.sum()
df.loc["Total", "Ax"] = df.Ax.sum()
df.loc["Total", "Ay"] = df.Ay.sum()

df['xNA'] = df['x']-xNA

df['$A{x_{NA}^2}$'] = (df['Area']*(df['xNA']**2))
df['$I_{locyy}$'] = (df['D']*(df['B']**3)/12)
df['Iyy'] = df['$A{x_{NA}^2}$'] + df['$I_{locyy}$']
df.loc["Total", "Iyy"] = df.Iyy.sum()
Iyy = df.Iyy.sum()
df.loc["Total", "$I_{locyy}$"] = df['$I_{locyy}$'].sum()
df.loc["Total", "$A{x_{NA}^2}$"] = df['$A{x_{NA}^2}$'].sum()


df['yNA'] = df['y']-yNA
df['$A{y_{NA}^2}$'] = (df['Area']*(df['yNA']**2))
df['$I_{locxx}$'] = (df['B']*(df['D']**3)/12)
df['Ixx'] = df['$A{y_{NA}^2}$'] + df['$I_{locxx}$']
df.loc["Total", "Ixx"] = df.Ixx.sum()
Ixx = df.Ixx.sum()
df.loc["Total", "$I_{locxx}$"] = df['$I_{locxx}$'].sum()
df.loc["Total", "$A{y_{NA}^2}$"] = df['$A{y_{NA}^2}$'].sum()

## df2 is used later for the cross girder bending




# .map("{:.1E}".format)
df = df.replace(np.nan, '', regex=True)
df1 = df.replace(np.nan, '', regex=True)



df1['$A{x_{NA}^2}$']= df1['$A{x_{NA}^2}$'].map("{:.2E}".format)
df1['$I_{locyy}$'] = df1['$I_{locyy}$'].map("{:.2E}".format)
df1['Iyy'] = df1['Iyy'].map("{:.2E}".format)
df1['$A{y_{NA}^2}$']= df1['$A{y_{NA}^2}$'].map("{:.2E}".format)
df1['$I_{locxx}$'] = df1['$I_{locxx}$'].map("{:.2E}".format)
df1['Ixx'] = df1['Ixx'].map("{:.2E}".format)




pd.set_option('display.float_format', lambda x: '%.0f' % x 
                      if (x == x and x*10 % 10 == 0) 
                      else ('%.0f' % x if (x == x and x*100 % 10 == 0)
                      else '%.0f' % x))



In [8]:
geom = df1[['B', 'D', 'x', 'y']].copy()
cols = geom.columns[geom.dtypes.eq('object')]
geom[cols] = geom[cols].apply(pd.to_numeric, errors='coerce')
geom['Origin_x'] = geom['x'] - 0.5* geom['B']
geom.loc[geom['y'] < 0, 'Origin_y'] = float(t_w/2)
geom.loc[geom['y'] >= 0, 'Origin_y'] = -geom['y'] - 0.5*geom['D']
geom.drop(geom.tail(1).index,inplace=True) # drop last n rows

page_ratio = 500/max(geom['B'])

x_r = geom['Origin_x']*page_ratio
y_r = geom['Origin_y']*page_ratio
B_r = geom['B']*page_ratio
D_r = geom['D']*page_ratio
Web_r = float(Web_width)*page_ratio
h_w = float(h)*page_ratio


#x_r2 = abs(min(geom['Origin_x']))
# x_r3 = list(x_r + x_r2)
# max_x = max(B_r)
# min_y = min(y_r)
# canvas_y = max(D_r)+float(t_w)
# canvas_x = max(B_r)

# canvas = Canvas(width=max_x, height=canvas_y)

# canvas.layout.width = '100%'
# canvas.layout.height = 'auto'

# canvas.fill_style = '#808080'
# canvas.translate(0,-min_y/2)
# canvas.scale(0.5,y=None)
# canvas.fill_rects(x_r3,y_r,B_r,D_r)
# canvas

In [9]:
canvas_y = 2*max(D_r)+float(t_w)
canvas_x = max(B_r)

de = draw.Drawing(canvas_x, canvas_y, origin='center', displayInline=False)


for x1, y1, b1, d1 in zip(x_r, y_r, B_r, D_r):
    r = draw.Rectangle(x1,y1,b1,d1, fill='#808080', stroke_width=1, stroke='black')
    de.append(r)   

display(de)




In [10]:
# ds = draw.Drawing(Web_r, h_w, origin='center', displayInline=False)
# # r2 = draw.Rectangle(-Web_r/2,0,Web_r, h_w, fill='#808080')
# # ds.append(r2) 

# for x1, b1 in zip(x_r, B_r):
#     r2 = draw.Rectangle(x1,0,b1,h_w, fill='none', stroke_width=1, stroke='black')
#     ds.append(r2) 

# ds


In [11]:
display(df1)

,name,B,D,x,y,Area,Ax,Ay,xNA,$A{x_{NA}^2}$,$I_{locyy}$,Iyy,yNA,$A{y_{NA}^2}$,$I_{locxx}$,Ixx
0,Inner 1,25,215,-300,120,5375,-1612500,645000,-300,4.84E+08,2.80E+05,4.84E+08,120,7.74E+07,2.07E+07,9.81E+07
1,Outer 2,25,215,-300,-120,5375,-1612500,-645000,-300,4.84E+08,2.80E+05,4.84E+08,-120,7.74E+07,2.07E+07,9.81E+07
2,Brg Inner 3,30,215,0,120,6450,0,774000,0,0.00E+00,4.84E+05,4.84E+05,120,9.29E+07,2.48E+07,1.18E+08
3,Brg Outer 4,30,215,0,-120,6450,0,-774000,0,0.00E+00,4.84E+05,4.84E+05,-120,9.29E+07,2.48E+07,1.18E+08
4,Inner 5,25,215,300,120,5375,1612500,645000,300,4.84E+08,2.80E+05,4.84E+08,120,7.74E+07,2.07E+07,9.81E+07
5,Outer 6,25,215,300,-120,5375,1612500,-645000,300,4.84E+08,2.80E+05,4.84E+08,-120,7.74E+07,2.07E+07,9.81E+07
6,Web,1400,25,0,0,35000,0,0,0,0.00E+00,5.72E+09,5.72E+09,0,0.00E+00,1.82E+06,1.82E+06
Total,,,,,,69400,0,0,,1.94E+09,5.72E+09,7.65E+09,,4.95E+08,1.34E+08,6.30E+08


Summary of section properties:

In [12]:
%%render params 1
Iyy = Iyy*mm**4
Ixx = Ixx*mm**4
A = A*mm**2

<IPython.core.display.Latex object>

## Tension field action <a name="a"></a>
Prior to collapse, vierendeel type action within a web panel causes additional forces within the stiffeners that border the web panel. If the web panels are subject to shear buckling, the stiffeners can contribute to improving the resistance of the girder near failure.   
![Figure 2](figure2.jpg)  
  

### British standard method  

Axial force due to tension field action in accordance with 9.13.3.2;  
Tension field action should be assumed to occur in any web panel when the average shear stress in the web panel, τ (from 9.5.1) is greater than $τ_{O}$, given by:

$ \tau_O = 3.6E(1+\frac{b}{a})(\frac{t_w}{b})^2 \sqrt{1- \frac{\sigma_1}{2.9E}(\frac{b}{t_w})^2}$       

when $ \sigma_1 < 2.9E(\frac{t_w}{b})^2$ or  

$ \tau_O = 0 $

when $ \sigma_1 \geq 2.9E(\frac{t_w}{b})^2$




firstly, define $\sigma_1$ the average longitudinal stress in the panel, taken as positive for compression.  

In [13]:
%%render precision 1

sigma_1 = (sigma_Tweb + sigma_Bweb)/2

<IPython.core.display.Latex object>

In [14]:
%%render precision 1
sigma_1limit = 2.9*E*(t_w/b)**2

<IPython.core.display.Latex object>

In [15]:
%%render precision 1

if sigma_1 < sigma_1limit: tau_O = 3.6*E*(1+(b/a)**2)*((t_w/b)**2)*sqrt(1- sigma_1/(2.9*E) *(b/t_w)**2)
elif sigma_1 >= sigma_1limit: tau_O=0  
    

<IPython.core.display.Latex object>

Tension field action should be assumed to cause a compressive force $F_{tw}$ in the adjacent transverse stiffener over its entire length, given by:

In [16]:
%%render 1
F_tw = min((tau - tau_O)*t_w*a,(tau - tau_O)*t_w*l_s).prefix('k')


<IPython.core.display.Latex object>

In [17]:
%%render 1
if F_tw > 0: F_tw
elif F_tw <= 0: F_tw =0

<IPython.core.display.Latex object>

There is a net tensile force in the section, therefore the effect of tension field action is zero. this makes sense, as the web is subject to a net tension, therefore plate buckling is not an issue.

### Eurocode method




## Destabilising effect of the web

In order to resist buckling of the web plate the effective stiffener section should be assumed to carry, along its centroidal axis, a compressive force $F_wi$ given by Cl. 9.13.3.3: 

$F_{wi} = \frac{l_s^2}{a}t_w k_s \sigma_R$  
  
Where $k_s$ is determined using the parameter $\lambda$  

![Figure 1](figure1.jpg)


In [18]:
%%render 1
r_se = sqrt(Ixx/A)*mm
sigma_ys = (345*MPa)



<IPython.core.display.Latex object>

Annex G.13 gives formulae for determining the curves in Figure 24.

In [19]:
%%render 1
lamb = (l_s/r_se)*sqrt(sigma_ys/(355*MPa))

if lamb > 15:
    n = 0.008*3*(lamb - 15)
elif lamb <= 15:
    n=0

sigma_l = 0.5*((1+(1+n)*5700/(lamb**2))- sqrt((1+(1+n)*5700/(lamb**2))**2 - 22800/(lamb**2)))

k_s = 0.4 * sigma_l *(n +(1.7546e-4)*lamb**2)

<IPython.core.display.Latex object>

In [20]:
%%render params 1
#short
tau_r = min(tau,tau_O)
A_s = 0*mm**2 # Sum of the area of longitudinal stiffeners on the web
sigma_b = (max(abs(sigma_Tweb),abs(sigma_Bweb)))


<IPython.core.display.Latex object>

In [21]:
%%render 1
sigma_R = tau_r + (1 + A_s/(l_s * t_w)) * (sigma_1 + sigma_b/6)

<IPython.core.display.Latex object>

Now that we have all of the variables we can calculate $F_{wi}$:  

In [22]:
%%render 1
F_wi = (l_s*l_s/a)*t_w*k_s*sigma_R


<IPython.core.display.Latex object>

In [23]:
F_wi2 = int(F_wi)


$F_{wi}$ is to be applied as a compressive force about the centroid of the stiffener.

## Axial force due to transfer of load through a cross frame/beam;

The bearing stiffeners also resist bending of the diaphragm cross girders.

The vertical force is considered within the bearing reaction, however the bending moment due hog at the connection will cause the the stiffeners to bend.

In [24]:
df2 = df[df['name'].str.match('Brg')]
Ixx_Brg = (df2.Ixx.sum())*mm**4
Y_Brg = (df2.D.min())*mm




In [25]:
%%render 1

Ixx_Brg 

Zxx_Brg = Ixx_Brg/(Y_Brg)

M_ED = 300*kNm

sigma_c = M_ED/Zxx_Brg



<IPython.core.display.Latex object>

In [26]:
# !jupyter nbconvert Brg.ipynb
#!jupyter nbconvert --to html_toc --no-input Brg.ipynb
!jupyter nbconvert --to html --no-input Brg.ipynb --TemplateExporter.exclude_input=True --no-prompt 

# adding the bits at the ends aligns all of the text

# https://stackoverflow.com/questions/21151450/how-can-i-add-a-table-of-contents-to-a-jupyter-jupyterlab-notebook

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook Brg.ipynb to html
[NbConvertApp] Writing 297070 bytes to Brg.html


In [1]:
!jupyter nbconvert --to pdf --no-input Brg.ipynb

[NbConvertApp] WARNING | Config option `kernel_spec_manager_class` not recognized by `NbConvertApp`.
[NbConvertApp] Converting notebook Brg.ipynb to pdf
[NbConvertApp] Writing 32790 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] CRITICAL | xelatex failed: ['xelatex', './notebook.tex', '-quiet']
This is XeTeX, Version 3.14159265-2.6-0.999992 (TeX Live 2020) (preloaded format=xelatex)
 restricted \write18 enabled.
entering extended mode
(./notebook.tex
LaTeX2e <2020-02-02> patch level 5
L3 programming layer <2020-03-06>
(/usr/local/texlive/2020/texmf-dist/tex/latex/base/article.cls
Document Class: article 2019/12/20 v1.4l Standard LaTeX document class
(/usr/local/texlive/2020/texmf-dist/tex/latex/base/size11.clo))
(/usr/local/texlive/2020/texmf-dist/tex/latex/tcolorbox/tcolorbox.sty
(/usr/local/texlive/2020/texmf-dist/tex/latex/pgf/basiclayer/pgf.sty
(/usr/local/texlive/2020/texmf-dist/tex

    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/Users/TomMcGonagle/opt/anaconda3/lib/python3.8/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/Users/TomMcGonagle/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 340, in start
    self.convert_notebooks()
  File "/Users/TomMcGonagle/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 510, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/Users/TomMcGonagle/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 481, in convert_single_notebook
    output, resources = self.export_single_notebook(notebook_filename, resources, input_buffer=input_buffer)
  File "/Users/TomMcGonagle/opt/anaconda3/lib/python3.8/site-packages/nbconvert/nbconvertapp.py", line 410, in export_single_notebook
    output, resources = self.exporter.from_filename(notebook_filename, reso